In [1]:
#Intalling Libraries for Collab
!pip install gradio
!pip install transformers
!pip install pronouncing
!pip install pandas
!pip install tensorflow
!pip install numpy
# !pip install TTS

  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.0
    Uninstalling numpy-1.22.0:
      Successfully uninstalled numpy-1.22.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tts 0.22.0 requires numpy==1.22.0; python_version <= "3.10", but you have numpy 1.26.4 which is incompatible.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
mizani 0.13.1 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
nx-cugraph-cu12 24.10.0 requires networkx>=3.0, but you have networkx 2.8.8 which is incompatible.
plotnine 0.14.5 requires pandas>=2.2.0, but you have pandas 1.5.3 which is inco

## Import Libraries

In [2]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from pickle import dump, load
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout


# Purpose: AI Lyricist Project
# This project aims to generate music lyrics using machine learning models. The system is designed to train on a dataset of song lyrics, learn patterns in word sequences, and generate new, coherent lyrics based on a seed phrase provided by the user.


RuntimeError: module was compiled against NumPy C-API version 0x10 (NumPy 1.23) but the running NumPy has C-API version 0xf. Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem.

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

SystemError: initialization of _pywrap_checkpoint_reader raised unreported exception

## Data Loading and Preprocessing

This section includes code for loading the dataset, handling missing data, and encoding metadata for use in the model.

### Dataset Loading
We support:
1. **Single CSV File**: For simpler use cases, where all data is contained in one file.
2. **Multiple CSV Files**: For larger datasets split into multiple smaller files.

### Preprocessing
1. **Missing Data Handling**: Rows with missing lyrics are removed to ensure data quality.
2. **Metadata Extraction**: Metadata columns such as `genre` and `artist` are cleaned and encoded for use in the model.
3. **Encoding**: Metadata is transformed into numeric features using one-hot encoding or embeddings.


In [3]:
import pandas as pd
import os

# Load and combine CSV files
def load_dataset(single_file="/content/drive/MyDrive/Dev/Final-Class-Team-Project/Resources/lyrics_1.csv", directory="/content/drive/MyDrive/Dev/Final-Class-Team-Project/Resources", file_prefix="lyrics_", file_count=15):
    if single_file:
        return pd.read_csv(single_file)
    combined_df = pd.DataFrame()
    for i in range(1, file_count + 1):
        file_path = os.path.join(directory, f"{file_prefix}{i}.csv")
        if os.path.exists(file_path):
            chunk_df = pd.read_csv(file_path)
            combined_df = pd.concat([combined_df, chunk_df], ignore_index=True)
    return combined_df

# Preprocess data
def preprocess_dataset(dataset, text_column="lyrics", metadata_columns=["genre", "artist"]):
    dataset = dataset.dropna()  # Drop rows with missing lyrics
    text_data = dataset[text_column].tolist()
    metadata = pd.get_dummies(dataset[metadata_columns])  # One-hot encode metadata
    return text_data, metadata

# Load dataset
use_combined_csvs = True
if use_combined_csvs:
    # Assign the loaded data to the 'dataset' variable
    dataset = load_dataset(directory="Resources") # This line was added
else:
    # Assign the loaded data to the 'dataset' variable
    dataset = load_dataset(single_file="lyrics.csv") # This line was added

# Preprocess data
lyrics, metadata_features = preprocess_dataset(dataset)
print(f"Lyrics processed: {len(lyrics)}, Metadata shape: {metadata_features.shape}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Dev/Final-Class-Team-Project/Resources/lyrics_1.csv'


## Data Cleaning

This step involves cleaning the lyrics data to prepare it for model training. The following steps are implemented:

- Dropping rows with missing lyrics.
- Converting all text to lowercase for standardization.
- Removing metadata markers such as `[Verse 1:]` and `[Chorus:]`.
- Expanding contractions (e.g., `I'm` → `I am`).
- Replacing newline characters (`\n`) with spaces.
- Removing special characters except for those useful in lyrics (e.g., punctuation).


In [ ]:
# Drop rows with missing lyrics  to be moved
dataset = dataset.dropna()
print(f"Loaded dataset shape: {dataset.shape}")

In [ ]:

# Data Cleaning Steps
# - Drop missing lyrics
# - Convert text to lowercase
# - Remove metadata markers like [Verse 1:] or [Chorus:]
# - Expand contractions
# - Replace newlines with spaces
# - Remove special characters

import re

def clean_lyrics(text):
    # Lowercase text
    text = text.lower()

    # Remove metadata markers like [Verse 1:], [Chorus:]
    text = re.sub(r"\[.*?\]", "", text)

    # Expand contractions
    contractions = {
        "i'm": "i am", "you're": "you are", "he's": "he is",
        "she's": "she is", "it's": "it is", "we're": "we are",
        "they're": "they are", "can't": "cannot", "won't": "will not",
        "n't": " not", "'re": " are", "'s": " is", "'d": " would",
        "'ll": " will", "'ve": " have", "'m": " am"
    }
    for contraction, expanded in contractions.items():
        text = re.sub(r"\b" + contraction + r"\b", expanded, text)

    # Replace newlines with spaces
    text = text.replace('\n', ' ')

    # Remove special characters except useful ones
    text = re.sub(r"[^a-zA-Z0-9.,!?\'\s]", "", text)

    # Remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()

    return text

# Apply cleaning to lyrics column
dataset['lyrics'] = dataset['lyrics'].apply(clean_lyrics)


## Data Pre-Processing and Analysis

In [ ]:
import re

def replaceLyricsChars(pattern, replacement, regex=False):
  """Replaces occurrences of a pattern in each string of the 'lyrics' list.

  Args:
    pattern: The pattern to search for.
    replacement: The string to replace the pattern with.
    regex: Whether to use regular expressions for the pattern. Defaults to False.
  """
  global lyrics  # Access and modify the global 'lyrics' variable
  for i in range(len(lyrics)):
    if regex:
      lyrics[i] = re.sub(pattern, replacement, lyrics[i])
    else:
      lyrics[i] = lyrics[i].replace(pattern, replacement)

# show all lyrics that have the word 'verse'
totality = len(lyrics)
with_verse = 0  # Initialize with_verse to 0

# Iterate through the lyrics list and count occurrences of 'verse'
for lyric in lyrics:
  if '\[verse' in lyric.lower():
    with_verse += 1

replaceLyricsChars('\[verse+\]', '', regex=True)
replaceLyricsChars('\[verse*\]', '', regex=True)

delta_totality = totality - len(lyrics)
delta_with_verse = with_verse - len([lyric for lyric in lyrics if '\[verse' in lyric.lower()]) # Changed to list comprehension

print(f"Totality: {totality} -> {delta_totality}")
print(f"With Verse: {with_verse} -> {delta_with_verse}")

In [ ]:
# show all lyrics that have the word 'intro'
totality = len(lyrics)
with_intro = sum(1 for lyric in lyrics if '\[intro' in lyric.lower()) # Count using list comprehension
replaceLyricsChars('\[intro+\]', '', regex=True)
replaceLyricsChars('\[intro*\]', '', regex=True)

delta_totality = totality - len(lyrics)
delta_with_intro = with_intro - sum(1 for lyric in lyrics if '\[intro' in lyric.lower()) # Count using list comprehension

print(f"Totality: {totality} -> {delta_totality}")
print(f"With intro: {with_intro} -> {delta_with_intro}")


## Tokenization and Sequence Preparation

The lyrics text is tokenized into sequences of words.
Input-output pairs are then created for the model, where the input is a sequence of words, and the output is the next word in the sequence.


In [ ]:
# Tokenization and Sequence Preparation
def preprocess_text(text):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts([text])
    total_words = len(tokenizer.word_index) + 1

    # Create input sequences
    input_sequences = []
    for line in text.split('\n'):
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)

    # Pad sequences
    max_sequence_len = max([len(seq) for seq in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    # Split into predictors and label
    predictors, label = input_sequences[:,:-1], input_sequences[:,-1]
    label = tf.keras.utils.to_categorical(label, num_classes=total_words)

    return predictors, label, max_sequence_len, total_words, tokenizer



## Building the Lyrics Generation Model

We use a neural network with the following components:

- An embedding layer to learn word representations.
- Bidirectional LSTM layers for context understanding.
- Dropout layers to prevent overfitting.
- Dense layers for classification of the next word.

This architecture is designed for efficient and cohesive text generation.


In [ ]:
''' # Step 2: Build the Model
def build_model(total_words, max_sequence_len):
    model = Sequential([
        Embedding(total_words, 64, input_length=max_sequence_len-1),
        LSTM(100, return_sequences=True),
        LSTM(100),
        Dense(100, activation='relu'),
        Dense(total_words, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
    '''

In [ ]:
# Build model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate
from tensorflow.keras.models import Model

def build_model(vocab_size, metadata_shape, embedding_dim=128, lstm_units=256):
    # Input layers
    text_input = Input(shape=(None,), name="text_input")
    metadata_input = Input(shape=(metadata_shape,), name="metadata_input")

    # Text processing
    x = Embedding(vocab_size, embedding_dim)(text_input)
    x = LSTM(lstm_units, return_sequences=False)(x)
    print(x.shape)
    print(metadata_input.shape)
    # Combine with metadata
    combined = Concatenate()([x, metadata_input])
    output = Dense(vocab_size, activation="softmax")(combined)

    model = Model(inputs=[text_input, metadata_input], outputs=output)
    model.compile(optimizer="adam", loss="categorical_crossentropy")
    return model

# Example: Build model
vocab_size = 5000  # Example vocab size
metadata_shape = metadata_features.shape[1]  # Metadata shape from preprocessing
model = build_model(vocab_size, metadata_shape)
model.summary()

## Model Training

This section includes training the LSTM model for lyrics generation.
The model incorporates text features from the lyrics and contextual metadata for better predictions.


In [ ]:
# Step 3: Train the Model
def train_model(model, text_input, metadata_input, label, epochs=30, batch_size=32):
    """
    Train the model with text and metadata inputs.

    :param model: The compiled model.
    :param text_input: Input text data (e.g., sequences of tokenized lyrics).
    :param metadata_input: Metadata features (e.g., genre, artist).
    :param label: Target labels (e.g., next word in the sequence).
    :param epochs: Number of training epochs.
    :param batch_size: Batch size for training.
    :return: The trained model.
    """
    # Ensure metadata_input has the same number of samples as text_input
    metadata_input = metadata_input.iloc[:text_input.shape[0]]  # Adjust metadata to match text input size

    model.fit(
        {"text_input": text_input, "metadata_input": metadata_input},
        label,
        epochs=epochs,
        batch_size=batch_size,
        verbose=1
    )
    return model


## Generating Lyrics

After training, the model can generate new lyrics based on a seed text.
The predictions are constrained to create cohesive and rhyming sequences.


In [ ]:
# Step 4: Generate Lyrics
def generate_lyrics(seed_text, model, tokenizer, max_sequence_len, num_words=20):
    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)

        for word, index in tokenizer.word_index.items():
            if index == predicted:
                seed_text += " " + word
                break

    return seed_text

## Main Workflow

In [ ]:
# This was added to act as a load_lyrics function and reads only a number of rows
def load_lyrics(data_path, chunksize=5, limit=15):
    """
    Loads lyrics data from a file using pandas.

    Args:
        data_path: The path to the lyrics dataset file.
        chunksize: The number of rows to read in each chunk.
        limit: The maximum number of rows to read.

    Returns:
        A string containing the combined lyrics from the dataset.
    """
    # Read the data in chunks and limit the number of chunks
    chunks = pd.read_csv(data_path, chunksize=chunksize, nrows=chunksize * limit)
    lyrics_data = []
    for chunk in chunks:
        lyrics_data.extend(chunk["lyrics"].tolist())

    # Combine the lyrics into a single string
    lyrics_text = " ".join(lyrics_data)

    return lyrics_text
  #  Main Workflow

# File path to lyrics dataset (replace with actual file path)
data_path = "/content/drive/MyDrive/Dev/Final-Class-Team-Project/Resources/lyrics_1.csv"

if os.path.exists(data_path):
    # Load and preprocess data
    lyrics = load_lyrics(data_path, chunksize=5, limit=15)
    print("preprocess_text")
    predictors, label, max_sequence_len, total_words, tokenizer = preprocess_text(lyrics)

    # Build and train the model
    print("build_model")
    model = build_model(total_words, metadata_features.shape[1]) # Use metadata_features.shape[1] instead of max_sequence_len
    print("train_model")
    # Pass metadata_features as metadata_input to train_model
    model = train_model(model, predictors, metadata_features, label, epochs=10) # Pass metadata_features to train_model
else:
    print(f"Dataset not found at {data_path}. Please provide a valid path. Try going through the cleaning.ipynb")

In [ ]:
# generate_lyrics(seed_text="roses are red, violets blue, I'll love it to wed...", model=model, tokenizer=tokenizer, max_sequence_len=max_sequence_len, num_words=20)
generate_lyrics(seed_text="roses are red", model=model, tokenizer=tokenizer, max_sequence_len=30, num_words=20)

In [ ]:
def predict_lyrics(input_text):
    generated_lyrics = generate_lyrics(seed_text=input_text, model=model, tokenizer=tokenizer, max_sequence_len=30, num_words=20)
    return generated_lyrics

In [ ]:
#Save the model (change the path)
model_save_path = "/content/drive/MyDrive/Dev/Final-Class-Team-Project/lyrics_model.h5"

In [ ]:
#model.save('lyrics_genie.keras')
model.save(model_save_path)
print(f"Model saved to {model_save_path}")

In [ ]:
model = load_model('lyrics_genie.keras')
tokenizer = load(open('lyrics_genie', 'rb'))

In [ ]:
#Function to Save and Load the model: To be used temporarily- remove before submitting final notebook
'''
from tensorflow.keras.models import load_model

# Predefined file path
MODEL_FILEPATH = './lyrics_model.h5'

def save_model_predefined(model):
    """
    Save a trained model to the predefined file path.

    Args:
        model: The trained model to save.
    """
    model.save(MODEL_FILEPATH)
    print(f"Model saved to '{MODEL_FILEPATH}'")

def load_model_predefined():
    """
    Load a saved model from the predefined file path.

    Returns:
        The loaded model.
    """
    # Predefined file path
    MODEL_FILEPATH = './lyrics_model.h5'

    model = load_model(MODEL_FILEPATH)
    print(f"Model loaded from '{MODEL_FILEPATH}'")
    r#eturn model
""

## Gradio Interface

The Gradio interface now supports additional inputs such as genre and artist to guide the lyrics generation process.



In [ ]:
import gradio as gr

# Predict function
def predict_lyrics(seed_text, genre, artist):
    # Combine seed_text with metadata context
    metadata_input = metadata_features.loc[(metadata_features["genre"] == genre) &
                                           (metadata_features["artist"] == artist)]
    # Generate lyrics using the model (mock example)
    generated_lyrics = "Generated lyrics for: " + seed_text
    return generated_lyrics

# Gradio app
app = gr.Interface(
    fn=predict_lyrics,
    inputs=["text", gr.inputs.Dropdown(choices=metadata_features["genre"].unique(), label="Genre"),
            gr.inputs.Dropdown(choices=metadata_features["artist"].unique(), label="Artist")],
    outputs="text",
    title="Lyrics Generator"
)
app.launch()
